In [1]:
import os
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

conf = (
    SparkConf()
    .setAppName("knmi daily weather to bronze")
    .set("spark.hadoop.fs.s3a.endpoint", "http://192.168.86.192:9000")
    .set("spark.hadoop.fs.s3a.access.key", os.getenv('MINIO_ROOT_USER'))
    .set("spark.hadoop.fs.s3a.secret.key", os.getenv('MINIO_ROOT_PASSWORD'))
    .set("spark.hadoop.fs.s3a.path.style.access", True)
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.driver.memory", "8g")
    .set("spark.executor.memory", "8g")
    .set("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") 
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
)
sc = SparkContext(conf=conf).getOrCreate()
sc.setLogLevel("ERROR")
spark = SparkSession(sc).builder.getOrCreate()


21/09/09 19:07:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/09/09 19:07:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.read.format('delta').load('s3a://silver-knmi/daggegevens')

In [16]:
from pyspark.sql.functions import desc, col, avg
df.groupBy('date').agg(avg('maximum_temperature').alias('avg_max_temp')).sort(desc('avg_max_temp')).show()


+----------+------------------+
|      date|      avg_max_temp|
+----------+------------------+
|2019-07-25| 37.36470588235295|
|2019-07-24| 36.16176470588235|
|2018-07-27| 35.56470588235294|
|2019-07-26| 35.46176470588235|
|2018-07-26| 35.12647058823529|
|1990-08-04| 34.33461538461538|
|1959-07-09|              34.2|
|2018-08-07| 34.18529411764706|
|1911-07-28|             34.16|
|2006-07-19|34.025714285714294|
|2010-07-09| 33.76470588235294|
|1994-08-04| 33.75333333333334|
|2020-08-08| 33.75294117647059|
|2009-08-20| 33.58529411764707|
|2014-07-19| 33.44117647058823|
|2019-06-25| 33.33529411764706|
|1990-08-03|33.323076923076925|
|2013-08-02|33.188235294117646|
|1994-07-24| 33.12333333333333|
|1947-06-27|             33.12|
+----------+------------------+
only showing top 20 rows

